In [ ]:
!python3 --version

In [ ]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade --user

In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

###  Build a Simple Pipeline

In [ ]:
import kfp
from kfp import dsl

def add_two_numbers(a, b):
    return dsl.ContainerOp(
        name = 'calculate_sum',
        image = 'python:3.6.8',
        command =['python','-c'],
        arguments = ['with open("/tmp/results.txt","a") as file: file.write(str({} + {}))'.format(a,b)],
        file_outputs={
            'data':'/tmp/results.txt',
        }
    )

In [ ]:
def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library /bash:4.4.23',
        command=['sh','-c'],
        arguments=['echo "Result: {}"'.format(text)]
    )

In [ ]:
@dsl.pipeline(
    name='Calculate sum pipeline',
    description='Calculate sum of numbers and prints the result.'
)

def calculate_sum(
    a=5,
    b=12,
    c=7,
    d=20
):

    sum1 = add_two_numbers(a,b)
    sum2 = add_two_numbers(c,d)
    sum = add_two_numbers(sum1.output, sum2.output)
    
    echo_task = echo_op(sum.output)

In [ ]:
kfp.compiler.Compiler().compile(calculate_sum, 'calculate-sum-pipeline.zip')

In [ ]:
!ls -al ./calculate-sum-pipeline.zip

In [ ]:
!unzip -o ./calculate-sum-pipeline.zip

In [ ]:
!pygmentize pipeline.yaml

### Deploy pipeline

In [ ]:
pipeline_func = calculate_sum

In [ ]:
experiment_name = 'python-functions'

#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

In [ ]:
client = kfp.Client()

experiment = client.create_experiment('kubeflow_pipeline_by_jupyter')

run_result = client.run_pipeline(
    experiment_id=experiment.id,
    job_name='calculate-sum-pipeline',
    pipeline_package_path='calculate-sum-pipeline.zip', 
    )
